# Set the Enviroment

In [1]:
import json
import os

# Rutas a los archivos de secrets
file_path_openai = r"C:\Users\********************\AppData\*********\**********\UserSecrets\************************************\secrets.json"
file_path_serper = r"C:\Users\********************\AppData\**********\**********\UserSecrets\************************************\secrets.json"

# Cargar OpenAI key
with open(file_path_openai, encoding='utf-8-sig') as f:
    openai_data = json.load(f)
openai_api_key = openai_data.get("ai_agents_openai")

# Cargar Serper key
with open(file_path_serper, encoding='utf-8-sig') as f:
    serper_data = json.load(f)
serper_api_key = serper_data.get("serper_api")

# Establecer como variables de entorno
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["SERPER_API_KEY"] = serper_api_key

# Confirmar
#print("🔑 OpenAI Key cargada:", openai_api_key[:5], "...")
#print("🔑 Serper Key cargada:", serper_api_key[:5], "...")


In [2]:
# import libraries
from crewai import Agent, Crew, Process, Task
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

C:\Users\*****************\********\Local\Programs\Python\********\lib\site-packages\**********\fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [3]:
# define the LLM
llm = ChatOpenAI(model_name = 'gpt-4o', temperature = 0.5)

# User inputs

In [4]:
# call for inputs: Interviewer, company, job position, job descriptions:
interviewer = input('Enter the name of the interviewer (if unknown include None): ')
company = input('Enter the name of the company: ')
job_position = input('Enter the job position: ')
job_description = input('Enter the job description: ')

Enter the name of the interviewer (if unknown include None):  Matthias Glöel
Enter the name of the company:  Beat81
Enter the job position:  Data Analytics Working Student
Enter the job description:  About the Role As a Working Student in our Analytics Team, you will work closely with our Tech, Product, Marketing and Operations team to support our daily operations with analytical insights and help us to grow. You will be joining a dynamic, fast-paced start-up where you’ll have ownership and impact from day one!  Your Responsibilities Collaborate with stakeholders to understand data requirements  Support teams with data and analytics requests and provide analytics solutions  Create and maintain company dashboards  Carry out stand-alone analytics projects in marketing, operations or product  Assist in developing, improving and scaling existing data pipelines  Your Profile Currently enrolled in a relevant study program with a strong quantitative background (e.g. mathematics, computer scie

# Research Crew

## Research AI Agent

In [5]:
# create research agent
research_agent = Agent(
    role = 'Research Agent',
    goal = 'conduct in-depth research',
    backstory = """As a Research Speciallist, your mission is to uncover detailed and relevant information.
    The ultimate goal is to meet the expectations of the task""",
    llm = llm,
    tools= [ScrapeWebsiteTool(), SerperDevTool()]
)

## Research Task

In [6]:
# define the task about researching the company
research_company_task = Task(
    description = f"""
    Research about {company}, their position in the sector and recent industry trends. Agent=Focus on recent news and developments from last 12 month""",
    expected_output = f"""A comprehensive report in Markdown with bulled point about {company} and it's industry""",
    agent = research_agent
)

In [7]:
# Research Interviewer Task
research_person_task = Task(
    description = f'Research about {interviewer} with a focus on their professional career',
    expected_output = f"""A comprehensive report in Markdown with bullet point about {interviewer}""",
    agent = research_agent
)

## AI Coach + Interview Prep

In [8]:
# Create a AI Interview Coach
coach_agent = Agent(
    role = 'Interview Coach',
    goal = f'I help the user prepare for the job interview for job {job_position} by grading the relevance of the answer',
    backstory = f'You are an expert in job interviews in companies like {company}',
    llm = llm
)

In [9]:
# Prepare the list of questions
define_questions_task = Task(
    description = f"""
    Define a list a of questions to prepare the user for the job {job_position} based on the company {company}, job description {job_description} and interviewer {interviewer}""",
    expected_output = f"""A list of 20 questions to ask the user in Markdown format that cover: Culture and team fit, Job position Fit, Background and ways of working, Growth Mindset""",
    agent = coach_agent,
    context = [research_company_task, research_person_task]
)

## Assemble the crew

In [10]:
crew = Crew(
    agents = [research_agent, coach_agent],
    tasks = [research_company_task, research_person_task, define_questions_task],
    verbose = True,
    process = Process.sequential
)

In [11]:
# kickoff
result = crew.kickoff({'topic': 'Write a list of question to prepare for the interview'})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 92ea725a-8123-4673-84ad-cd291148e559                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Agent                                                                                          │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Research about Beat81, their position in the sector and recent industry trends. Agent=Focus on recent      │
│  news and developments from last 12 month                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Agent                                                                                          │
│                                                                                                                 │
│  Thought: Thought: To provide a comprehensive report on Beat81, their position in the sector, and recent        │
│  industry trends, I need to gather recent news and developments about Beat81 from the last 12 months. I will    │
│  start by searching for the latest information about Beat81 using the internet search tool.                     │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"Beat81 recent news 2023\"}"                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Interview Coach                                                                                         │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Define a list a of questions to prepare the user for the job Data Analytics Working Student based on the   │
│  company Beat81, job description About the Role As a Working Student in our Analytics Team, you will work       │
│  closely with our Tech, Product, Marketing and Operations team to support our daily operations with analytical  │
│  insights and help us to grow. You will be joining a dynamic, fast-paced start-up where you’ll have ownership   │
│  and impact from day one!  Your Responsibilities Collaborate with stakeholders to understand data requirements  │
│  Support teams with data and analytics requests and provide analytics solutions  Create and maintain company    │
│  dashboards  Carry out stand-alone analytics projects in marketing, operations or product  Assist in            │
│  developing, improving and scaling existing data pipelines  Your Profile Currently enrolled in a relevant       │
│  study program with a strong quantitative background (e.g. mathematics, computer science, data science,         │
│  business informatics etc.)  Good knowledge of Google Sheets  Good Knowledge of SQL  Good Knowledge of Python   │
│  Knowledge of common data visualization tools (e.g. Tableau, Lookerstudio etc.)  Knowledge of dbt, GCP,         │
│  BigQuery is a plus  Fluent in English  OUR OFFER  WALK THE TALK: Unlimited BEAT81 workouts for you and one     │
│  family member/friend: we guarantee that you will get fit working here :)  YOUR DEVICE: Everyone has their own  │
│  preferences when it comes to technology - whether it’s Mac or Windows - you choose your laptop. Additionally,  │
│  we provide Bose noise-canceling headphones  HYBRID WORK: We have a beautiful office in the heart of Berlin     │
│  (Weinmeisterstraße) with free drinks, healthy snacks and fruits. We like to spend time there together, but     │
│  you are also welcome to work from home  FLEXIBLE WORKING HOURS: Early bird or late riser? Shape your workday   │
│  to align with your personal commitments and preferences in coordination with your team and the business needs  │
│  TEAM RITUALS: Shared experiences build a strong team culture. That’s why we have our usuals such as regular    │
│  all hands meetings, Aperol Thursdays and joint workouts - there is something for everyone  TEAM EVENTS: Our    │
│  summer and Christmas parties have become as legendary as our annual off-sites and the perfect occasion to      │
│  spend quality time with team members and interviewer Matthias Glöel                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Interview Coach                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # Interview Preparation Questions for Data Analytics Working Student at Beat81                                 │
│                                                                                                                 │
│  ## Culture and Team Fit                                                                                        │
│  1. How do you align with Beat81's mission to provide smarter workouts and support community-driven fitness     │
│  experiences?                                                                                                   │
│  2. Can you describe a time when you contributed to a team culture in a fast-paced environment?                 │
│  3. How do you prioritize team rituals and shared experiences in a hybrid work setting?                         │
│  4. How do you see yourself contributing to a supportive community at Beat81?                                   │
│  5. What aspects of Beat81's culture, such as flexibility and team events, appeal to you the most?              │
│                                                                                                                 │
│  ## Job Position Fit                                                                                            │
│  6. How do you plan to collaborate with different teams like Tech, Product, Marketing, and Operations to        │
│  provide analytical insights?                                                                                   │
│  7. Can you discuss your experience with creating and maintaining dashboards?                                   │
│  8. How do you approach stand-alone analytics projects, and can you provide an example?                         │
│  9. What is your experience with developing and improving data pipelines?                                       │
│  10. How do you handle multiple data and analytics requests from various stakeholders?                          │
│                                                                                                                 │
│  ## Background and Ways of Working                                                                              │
│  11. What relevant study program are you currently enrolled in, and how does it prepare you for this role?      │
│  12. Can you discuss your proficiency with Google Sheets, SQL, and Python?                                      │
│  13. What data visualization tools are you familiar with, and how have you used them in the past?               │
│  14. How would you rate your knowledge of dbt, GCP, and BigQuery, and how do you plan to enhance these skills?  │
│  15. How do you ensure effective communication and collaboration in a diverse and dynamic team?                 │
│                                                                                                                 │
│  ## Growth Mindset                                                                                              │
│  16. How do you stay updated with the latest trends in data analytics and the fitness industry?                 │
│  17. Can you share an example of a time you learned a new skill or tool that significantly impacted your work?  │
│  18. How do you approach challenges and setbacks in a p

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ee1dc4b9-5fd2-49b3-bda1-19e27848af38                                                                     │
│  Agent: Interview Coach                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 92ea725a-8123-4673-84ad-cd291148e559                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  # Interview Preparation Questions for Data Analytics Working Student at Beat81                                 │
│                                                                                                                 │
│  ## Culture and Team Fit                                                                                        │
│  1. How do you align with Beat81's mission to provide smarter workouts and support community-driven fitness     │
│  experiences?                                                                                                   │
│  2. Can you describe a time when you contributed to a team culture in a fast-paced environment?                 │
│  3. How do you prioritize team rituals and shared experiences in a hybrid work setting?                         │
│  4. How do you see yourself contributing to a supportive community at Beat81?                                   │
│  5. What aspects of Beat81's culture, such as flexibility and team events, appeal to you the most?              │
│                                                                                                                 │
│  ## Job Position Fit                                                                                            │
│  6. How do you plan to collaborate with different teams like Tech, Product, Marketing, and Operations to        │
│  provide analytical insights?                                                                                   │
│  7. Can you discuss your experience with creating and maintaining dashboards?                                   │
│  8. How do you approach stand-alone analytics projects, and can you provide an example?                         │
│  9. What is your experience with developing and improving data pipelines?                                       │
│  10. How do you handle multiple data and analytics requests from various stakeholders?                          │
│                                                                                                                 │
│  ## Background and Ways of Working                                                                              │
│  11. What relevant study program are you currently enrolled in, and how does it prepare you for this role?      │
│  12. Can you discuss your proficiency with Google Sheets, SQL, and Python?                                      │
│  13. What data visualization tools are you familiar with, and how have you used them in the past?               │
│  14. How would you rate your knowledge of dbt, GCP, and BigQuery, and how do you plan to enhance these skills?  │
│  15. How do you ensure effective communication and collaboration in a diverse and dynamic team?                 │
│                                                                                                                 │
│  ## Growth Mindset                                                                                              │
│  16. How do you stay updated with the latest trends in data analytics and the fitness industry?                 │
│  17. Can you share an example of a time you learned a 

## Exporting

In [22]:
# Check the crew outcome
display(Markdown(result.tasks_output[2].raw))

```markdown
# Interview Questions for Data Analytics Working Student at Beat81

## Culture and Team Fit
1. How do you feel about working in a fast-paced start-up environment like Beat81?
2. Can you describe a time when you had to adapt to a new team culture? What was challenging and how did you overcome it?
3. How do you think you can contribute to the team rituals and events at Beat81?
4. How do you handle feedback and collaboration when working with cross-functional teams?
5. What attracts you to the fitness industry, and how do you see yourself fitting into Beat81's mission and values?

## Job Position Fit
6. How would you approach collaborating with stakeholders to understand their data requirements at Beat81?
7. Can you give an example of a stand-alone analytics project you have completed? What was the outcome?
8. What experience do you have with creating and maintaining dashboards? Which tools do you prefer and why?
9. How do you prioritize analytics requests from different teams such as marketing, operations, and product?
10. How familiar are you with dbt, GCP, and BigQuery, and how would you leverage these tools in your role?

## Background and Ways of Working
11. Can you describe your experience with SQL and Python in academic or professional settings?
12. How have you used Google Sheets in past projects, and what advanced features are you comfortable with?
13. How do you stay updated with the latest trends and technologies in data analytics?
14. Describe a time when you improved or scaled a data pipeline. What were the challenges and solutions?
15. How do you manage your time and tasks when working on multiple projects simultaneously?

## Growth Mindset
16. Can you share an experience where you faced a significant challenge in data analytics and how you overcame it?
17. How do you approach learning new data visualization tools such as Tableau or Lookerstudio?
18. What are your long-term career goals in data analytics, and how do you see this role at Beat81 helping you achieve them?
19. How do you handle situations where you are required to learn new skills or tools quickly?
20. Can you discuss a time when you proactively sought feedback to improve your work or skills?

```

This list of questions is designed to assess the candidate's fit within Beat81's culture and team, their suitability for the job position, their background and ways of working, and their growth mindset. These questions aim to provide a comprehensive understanding of the candidate's abilities and potential contributions to Beat81.

In [12]:
# Export each of the tasks output
# exporting the company report
file_name = f'{company}_report.txt'

# Open the file in write mode and write the content
with open(file_name, 'w') as file:
    file.write(result.tasks_output[0].raw)

# Export the interviewer report
file_name = f'{interviewer}_report.txt'

# Open the file in write mode and write the content
with open(file_name, 'w') as file:
    file.write(result.tasks_output[1].raw)

# Export the interview questions
file_name = f'{job_position}_questions.txt'

# Open the file in write mode and write the content
with open(file_name, 'w') as file:
    file.write(result.tasks_output[2].raw)


# Interview Prep

In [13]:
# import the questions from the directory
with open(f'{job_position}_questions.txt', 'r') as file:
    questions = file.read()

In [14]:
# see the questions
questions

"```markdown\n# Interview Preparation Questions for Data Analytics Working Student at Beat81\n\n## Culture and Team Fit\n1. How do you align with Beat81's mission to provide smarter workouts and support community-driven fitness experiences?\n2. Can you describe a time when you contributed to a team culture in a fast-paced environment?\n3. How do you prioritize team rituals and shared experiences in a hybrid work setting?\n4. How do you see yourself contributing to a supportive community at Beat81?\n5. What aspects of Beat81's culture, such as flexibility and team events, appeal to you the most?\n\n## Job Position Fit\n6. How do you plan to collaborate with different teams like Tech, Product, Marketing, and Operations to provide analytical insights?\n7. Can you discuss your experience with creating and maintaining dashboards?\n8. How do you approach stand-alone analytics projects, and can you provide an example?\n9. What is your experience with developing and improving data pipelines?\n

In [15]:
# AI agent that ask the question
interviewer_agent = Agent(
    role = 'Interviewer Agent',
    goal = f'Ask a question from {questions} to the user',
    backstory = f'You are an expert in job interviews in companies like {company}',
    llm = llm
)

In [16]:
# Interview task
interview_prep_task = Task(
    description = f'Ask a question from {questions} to the user',
    expected_output = f'Ask only one question at a time',
    agent = interviewer_agent,
    human_input = True
)

In [17]:
# Feedback Task
feedback_task = Task(
    description = """Give feedback to the user on it's answer""",
    expected_output = 'Markdown answer of what was good, could be improved and how o take it to the next level',
    agent = coach_agent,
    context = [interview_prep_task],
    human_input = True
)

## Interview Prep Crew

In [18]:
# Assemble the interview prep crew
crew = Crew(
    agents = [interviewer_agent, coach_agent],
    tasks = [interview_prep_task, feedback_task],
    verbose = True,
    process = Process.sequential
)

In [19]:
result = crew.kickoff({'topic': f'Ask me all questions from {questions}'})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5c9e5250-4656-4233-8f38-71f6e2b283cd                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Interviewer Agent                                                                                       │
│                                                                                                                 │
│  Task: Ask a question from ```markdown                                                                          │
│  # Interview Preparation Questions for Data Analytics Working Student at Beat81                                 │
│                                                                                                                 │
│  ## Culture and Team Fit                                                                                        │
│  1. How do you align with Beat81's mission to provide smarter workouts and support community-driven fitness     │
│  experiences?                                                                                                   │
│  2. Can you describe a time when you contributed to a team culture in a fast-paced environment?                 │
│  3. How do you prioritize team rituals and shared experiences in a hybrid work setting?                         │
│  4. How do you see yourself contributing to a supportive community at Beat81?                                   │
│  5. What aspects of Beat81's culture, such as flexibility and team events, appeal to you the most?              │
│                                                                                                                 │
│  ## Job Position Fit                                                                                            │
│  6. How do you plan to collaborate with different teams like Tech, Product, Marketing, and Operations to        │
│  provide analytical insights?                                                                                   │
│  7. Can you discuss your experience with creating and maintaining dashboards?                                   │
│  8. How do you approach stand-alone analytics projects, and can you provide an example?                         │
│  9. What is your experience with developing and improving data pipelines?                                       │
│  10. How do you handle multiple data and analytics requests from various stakeholders?                          │
│                                                                                                                 │
│  ## Background and Ways of Working                                                                              │
│  11. What relevant study program are you currently enrolled in, and how does it prepare you for this role?      │
│  12. Can you discuss your proficiency with Google Sheets, SQL, and Python?                                      │
│  13. What data visualization tools are you familiar with, and how have you used them in the past?               │
│  14. How would you rate your knowledge of dbt, GCP, and BigQuery, and how do you plan to enhance these skills?  │
│  15. How do you ensure effective communication and collaboration in a diverse and dynamic team?                 │
│                                                                                                                 │
│  ## Growth Mindset                                                                                              │
│  16. How do you stay updated with the latest trends in data analytics and the fitness industry?                 │
│  17. Can you share an example of a time you learned a new skill or tool that significantly impacted your work?  │
│  18. How do you approach challenges and setbacks in a project?                                                  │
│  19. What are your career goals, and how does this posi

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Interviewer Agent                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  How do you align with Beat81's mission to provide smarter workouts and support community-driven fitness        │
│  experiences?                                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 ## Final Result: How do you align with Beat81's mission to provide smarter workouts and support community-driven fitness experiences?
 

=====
## HUMAN FEEDBACK: Provide feedback on the Final Result and Agent's actions.
Please follow these guidelines:
 - If you are happy with the result, simply hit Enter without typing anything.
 - Otherwise, provide specific improvement requests.
 - You can provide multiple rounds of feedback until satisfied.
=====



...

 
Processing your feedback...


Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Interviewer Agent                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Current Task: Ask a question from ```markdown                                                                  │
│  # Interview Preparation Questions for Data Analytics Working Student at Beat81                                 │
│                                                                                                                 │
│  ## Culture and Team Fit                                                                                        │
│  1. How do you align with Beat81's mission to provide smarter workouts and support community-driven fitness     │
│  experiences?                                                                                                   │
│  2. Can you describe a time when you contributed to a team culture in a fast-paced environment?                 │
│  3. How do you prioritize team rituals and shared experiences in a hybrid work setting?                         │
│  4. How do you see yourself contributing to a supportive community at Beat81?                                   │
│  5. What aspects of Beat81's culture, such as flexibility and team events, appeal to you the most?              │
│                                                                                                                 │
│  ## Job Position Fit                                                                                            │
│  6. How do you plan to collaborate with different teams like Tech, Product, Marketing, and Operations to        │
│  provide analytical insights?                                                                                   │
│  7. Can you discuss your experience with creating and maintaining dashboards?                                   │
│  8. How do you approach stand-alone analytics projects, and can you provide an example?                         │
│  9. What is your experience with developing and improving data pipelines?                                       │
│  10. How do you handle multiple data and analytics requests from various stakeholders?                          │
│                                                                                                                 │
│  ## Background and Ways of Working                                                                              │
│  11. What relevant study program are you currently enrolled in, and how does it prepare you for this role?      │
│  12. Can you discuss your proficiency with Google Sheets, SQL, and Python?                                      │
│  13. What data visualization tools are you familiar with, and how have you used them in the past?               │
│  14. How would you rate your knowledge of dbt, GCP, and BigQuery, and how do you plan to enhance these skills?  │
│  15. How do you ensure effective communication and collaboration in a diverse and dynamic team?                 │
│                                                                                                                 │
│  ## Growth Mindset                                                                                              │
│  16. How do you stay updated with the latest trends in data analytics and the fitness industry?                 │
│  17. Can you share an example of a time you learned a new skill or tool that significantly impacted your work?  │
│  18. How do you approach challenges and setbacks in a p

 ## Final Result: Current Task: Ask a question from ```markdown
# Interview Preparation Questions for Data Analytics Working Student at Beat81

## Culture and Team Fit
1. How do you align with Beat81's mission to provide smarter workouts and support community-driven fitness experiences?
2. Can you describe a time when you contributed to a team culture in a fast-paced environment?
3. How do you prioritize team rituals and shared experiences in a hybrid work setting?
4. How do you see yourself contributing to a supportive community at Beat81?
5. What aspects of Beat81's culture, such as flexibility and team events, appeal to you the most?

## Job Position Fit
6. How do you plan to collaborate with different teams like Tech, Product, Marketing, and Operations to provide analytical insights?
7. Can you discuss your experience with creating and maintaining dashboards?
8. How do you approach stand-alone analytics projects, and can you provide an example?
9. What is your experience with devel

KeyboardInterrupt: Interrupted by user

In [20]:
# This is so you can use the 20 questions
# Assemble the interview prep crew
crew = Crew(
    agents = [interviewer_agent, coach_agent],
    tasks = [interview_prep_task, feedback_task],
    verbose = True,
    process = Process.hierarchical,
    manager_llm = ChatOpenAI(model_name = 'gpt-4o', temperature = 0),
    memory = True,
    planning = True
)

C:\Users\Sebastian Perez Vega\AppData\Roaming\Python\Python310\site-packages\chromadb\types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


In [ ]:
result = crew.kickoff({'topic': f'Ask me all questions from {questions}'})